In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
batches = ['leeuwarder_courant', 'telegraaf', 'trouw', 'volkskrant']

In [ ]:
import spacy
from spacy.pipeline import EntityRuler
from src.config import PATH_MODEL, PARAM
from src.spacy_helpers import serialize_batch
from src.geo_data import load_rest_countries, load_cbs_dutch_cities, parse_wiki_place_lists, load_capitals_from_wiki
from src.geo_data import create_geonames_datasets

In [ ]:
create_geonames_datasets(language='nl')

In [ ]:
cbs_data = load_cbs_dutch_cities()
topo = {
    'countries': load_rest_countries(language='nl', alts_json='alts.json'),
    'capitals' : [country_cap[1] for country_cap in load_capitals_from_wiki()],
    'places_uk': parse_wiki_place_lists('https://nl.wikipedia.org/wiki/Lijst_van_plaatsen_in_Engeland'),
    'places_nl': cbs_data.gemeentenaam.values,
    'places_fr': cbs_data.query("provincie == 'Friesland'").gemeentenaam.values,
    }

geo_ents = dict()
for label in topo:
    geo_ents[label] = [{'label': label, 'pattern': p} for p in topo[label]]

In [ ]:
nlp = spacy.load('nl', disable=['ner'])
ruler = EntityRuler(nlp)
for label in geo_ents:
    ruler.add_patterns(geo_ents[label])
nlp.add_pipe(ruler)
nlp.to_disk(PATH_MODEL)

In [ ]:
for batch in batches:
    serialize_batch(nlp, batch)